In [1]:
import tensorflow as tf
import numpy as np
import src

In [2]:
problem = src.SortingProblem()

In [3]:
network = src.PointerNetwork('cfg')

seqLen = network.maxTimeSteps
batchSize = network.batchSize

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [6]:
trainingParams = {}
trainingParams['batchSize'] = batchSize
trainingParams['length'] = seqLen

losses = []

saver = tf.train.Saver()
with tf.Session() as sess:
    # initialize variables
    sess.run(tf.global_variables_initializer())
    
    # write out log for graph visualization
    writer = tf.summary.FileWriter('checkpoints/', sess.graph)

    for batch in range(100):
        # make current batch of training data
        inputs = problem.makeBatch(trainingParams)
        trainingInputs, targets = problem.makeTargets(inputs)
        
        # apply one training step
        _, loss = sess.run([network.trainOp, network.loss], 
                           feed_dict={network.train: True,
                                      network.rawInputs: inputs,
                                      network.targets: targets,
                                      network.targetInputs: trainingInputs[:,:-1,:]})
        
        # keep track of the loss values
        losses.append(loss)
        
        if batch % 100 == 0 or batch == 99:
            
            # generate a test batch and output its accuracy and also a sample
            print('Batch: ' + str(batch))
            newInputs = problem.makeBatch(trainingParams)
            output = sess.run(network.decoderOutputs, feed_dict={network.train: False,
                                                                 network.rawInputs: newInputs})
            print(newInputs[0, :, :].T[0])
            print(newInputs[0, :, :].T[0][np.array(output).T[0]])
            print(problem.accuracy(solution=np.array(output).T, batchArray=newInputs))
            
            # save checkpoint
            if batch > 0:
                saver.save(sess, 'checkpoints/SortingPointerNetwork', global_step=network.globalStep)

Batch: 0
[ 0.36729015  0.42409987  0.57016633]
[ 0.42409987  0.57016633  0.36729015]
0.12
Batch: 99
[ 0.72840221  0.57988386  0.33264906]
[ 0.33264906  0.57988386  0.72840221]
0.97


In [ ]:
import matplotlib.pyplot as plt

plt.semilogy(losses)
plt.show()